In [1]:
import pandas as pd  
import requests  
from bs4 import BeautifulSoup
import numpy as np
import re


# Initialize empty lists to store Device Names and Colors
device_names = []
colors = []
RAM_Sizes = []
ROM_Sizes = []
Display_Sizes = []
Battery_Capacities = []
prices = []

url = (
    "https://www.flipkart.com/search?q=mobiles&as=on&as-show=on&"
    "otracker=AS_Query_TrendingAutoSuggest_1_0_na_na_na&"
    "otracker1=AS_Query_TrendingAutoSuggest_1_0_na_na_na&"
    "as-pos=1&as-type=TRENDING&suggestionId=mobiles&"
    "requestId=b87edce2-2302-4ed2-87db-5e11533453f2&"
    "p%5B%5D=facets.brand%255B%255D%3DSAMSUNG&page=1"
    )

headers = ({'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'})

webpage = requests.get(url,headers)

soup = BeautifulSoup(webpage.text, 'html')

'''----------------------------------------------------Name Color--------------------------------------------------------------'''

product_list = soup.find_all('div', class_ ="KzDlHZ")
product_name = [product.text.strip() for product in product_list]

# Loop through the product list
for product in product_name:
    # Extract Device Name (everything before the first opening parenthesis)
    name_match = re.match(r'(.*?)(?=\s*\()', product)
    device_name = name_match.group(1).strip() if name_match else product.strip()
    device_names.append(device_name)
    
    # Extract Color (text between the opening and closing parenthesis, excluding GB info)
    color_match = re.search(r'\((.*?),\s*\d+\s*GB\)', product)
    # color = color_match.group(1).strip() if color_match else 'No Color Mentioned'
    color = color_match.group(1).strip() if color_match else None
    colors.append(color)

'''--------------------------------RAM ROM BAttery Display--------------------------------------'''

Features_list = soup.find_all('li',class_="J+igdf")
Features = [product.text.strip() for product in Features_list]

# Function to convert MB to GB
def convert_mb_to_gb(mb_value):
    return round(float(mb_value) / 1024, 4)  # Convert MB to GB with 4 decimal places

# Loop through each feature in the list
for feature in Features:
    # Extract RAM size (in GB or MB)
    ram_match_gb = re.search(r'(\d+)\s*GB RAM', feature)
    ram_match_mb = re.search(r'([\d.]+)\s*MB RAM', feature)
    if ram_match_gb:
        RAM_Sizes.append(int(ram_match_gb.group(1)))  # Append GB directly
    elif ram_match_mb:
        RAM_Sizes.append(convert_mb_to_gb(ram_match_mb.group(1)))  # Convert MB to GB and append
    
    # Extract ROM size (in GB or MB)
    rom_match_gb = re.search(r'(\d+)\s*GB ROM', feature)
    rom_match_mb = re.search(r'([\d.]+)\s*MB ROM', feature)
    if rom_match_gb:
        ROM_Sizes.append(int(rom_match_gb.group(1)))  # Append GB directly
    elif rom_match_mb:
        ROM_Sizes.append(convert_mb_to_gb(rom_match_mb.group(1)))  # Convert MB to GB and append
    
    # Extract Display size in inches
    display_match = re.search(r'\(([\d.]+)\s*inch\)', feature)
    if display_match:
        Display_Sizes.append(float(display_match.group(1)))  # Convert to float and append
    
    # Extract Battery capacity in mAh
    battery_match = re.search(r'(\d+)\s*mAh', feature)
    if battery_match:
        Battery_Capacities.append(int(battery_match.group(1)))  # Convert to int and append

'''---------------------------------Price-----------------------------'''
# Extracting all the product prices as text
price_list = soup.find_all('div', class_="Nx9bqj _4b5DiR")

# Extract numerical prices, removing ₹ and commas using regular expression
product_price = [product.text.strip() for product in price_list]

# Loop through the extracted prices
for price in product_price:
    # Remove ₹ and commas, and keep only numerical values
    price_clean = re.sub(r'[^\d]', '', price)  # Remove anything that is not a digit
    if price_clean:
        prices.append(int(price_clean))  # Convert to integer and append
    else:
        prices.append(None)  # If no valid price is found, append None

'''---------------------------------Price-----------------------------'''

# Create the DataFrame
df = pd.DataFrame({
    'Device Name': device_names,
    'Price (Rs.)': prices,
    'Color': colors,
    'RAM Size (GB)': RAM_Sizes,
    'ROM Size (GB)': ROM_Sizes,
    'Display Size (inches)': Display_Sizes,
    'Battery Capacity (mAh)': Battery_Capacities
})

# Display the DataFrame
print(df)

                    Device Name  Price (Rs.)            Color  RAM Size (GB)  \
0         SAMSUNG Galaxy A14 5G        10999         Dark Red              6   
1         SAMSUNG Galaxy A14 5G        10999      Light Green              6   
2         SAMSUNG Galaxy S23 5G        42999            Cream              8   
3            SAMSUNG Galaxy F05         6499    Twilight Blue              4   
4         SAMSUNG Galaxy A14 5G        10999            Black              6   
5         SAMSUNG Galaxy S23 5G        37999            Cream              8   
6         SAMSUNG Galaxy S23 5G        37999            Green              8   
7         SAMSUNG Galaxy S23 5G        42999         Lavender              8   
8         SAMSUNG Galaxy S23 5G        42999            Green              8   
9         SAMSUNG Galaxy S23 5G        37999    Phantom Black              8   
10        SAMSUNG Galaxy S23 5G        37999         Lavender              8   
11        SAMSUNG Galaxy S23 5G        4

In [28]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

# Initialize empty lists to collect data
names = []
prices = []
ratings = []
ratings_count = []
reviews_count = []
ram_list = []
rom_list = []
display_list = []
battery_list = []

# Function to extract ratings and reviews numbers
def extract_ratings_and_reviews(feedback):
    pattern = r'(\d{1,3}(?:,\d{3})*)\s*Ratings.*&\s*(\d{1,3}(?:,\d{3})*)\s*Reviews'
    match = re.search(pattern, feedback)
    if match:
        # Remove commas and convert to integers
        number_of_ratings = int(match.group(1).replace(',', ''))
        number_of_reviews = int(match.group(2).replace(',', ''))
        return number_of_ratings, number_of_reviews
    else:
        return None, None

# Function to convert MB to GB if applicable (for battery capacity in MB only)
def convert_mb_to_gb(mb_value):
    if mb_value.endswith("MB"):
        return round(int(mb_value.replace('MB', '').strip()) / 1024, 4)  # Convert MB to GB
    return int(mb_value)  # Return as integer if it's already in mAh

# Loop through pages from 1 to 17
for page in range(1,18):
    print(f"Scraping page {page}...")
    
    # Construct the page URL
    url = ("https://www.flipkart.com/search?q=mobiles&as=on&as-show=on&"
           "otracker=AS_Query_TrendingAutoSuggest_1_0_na_na_na&"
           "otracker1=AS_Query_TrendingAutoSuggest_1_0_na_na_na&"
           "as-pos=1&as-type=TRENDING&suggestionId=mobiles&" 
           "requestId=b87edce2-2302-4ed2-87db-5e11533453f2&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG&" 
           "p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&page={}").format(page)

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
    webpage = requests.get(url, headers=headers)

    # Check if request was successful
    if webpage.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {webpage.status_code}")
        continue

    soup = BeautifulSoup(webpage.text, 'html')

    for i in soup.find_all('div',class_ = "tUxRFH"):
        # Extract product name (handle missing data)
        product_name = i.find('div', class_="KzDlHZ")
        names.append(product_name.text.strip() if product_name else np.nan)

        # Extract product price and convert to integer (handle missing data)
        price_tag = i.find('div', class_="Nx9bqj _4b5DiR")
        prices.append(int(price_tag.text.replace('₹', '').replace(',', '').strip()) if price_tag else np.nan)

        # Extract product rating and convert to float (handle missing data)
        rating_tag = i.find('div', class_="XQDdHH")
        ratings.append(float(rating_tag.text.strip()) if rating_tag else np.nan)

        # Extract ratings_before and ratings_after using the function
        feedback_tag = i.find('span', class_="Wphh3N")
        feedback = feedback_tag.text if feedback_tag else None
        # Check if feedback is not None before calling the function
        if feedback:
            rating_value, review_value = extract_ratings_and_reviews(feedback)
        else:
            rating_value, review_value = None, None  # Handle missing feedback case
        ratings_count.append(rating_value)
        reviews_count.append(review_value)

        # Extract RAM, ROM, Display (inch), and Battery Capacity (mAh)
        text = i.find('ul', class_="G4BRas").text.strip()

        ram_match = re.search(r'(\d+)\s*GB\s*RAM', text)
        rom_match = re.search(r'(\d+)\s*GB\s*ROM', text)
        display_match = re.search(r'\((\d+(\.\d+)?)\s*inch\)', text)
        battery_match = re.search(r'(\d+)\s*(mAh|MB)', text)

        # Handle missing values and append to lists
        ram = float(ram_match.group(1)) if ram_match else np.nan
        rom = float(rom_match.group(1)) if rom_match else np.nan
        display = float(display_match.group(1)) if display_match else np.nan

        # Extract the battery capacity (only in mAh)
        if battery_match:
            battery = int(battery_match.group(1))  # Always keep it in mAh
        else:
            battery = np.nan  # Handle missing battery data

        # Append extracted values to the lists
        ram_list.append(ram)
        rom_list.append(rom)
        display_list.append(display)
        battery_list.append(battery)

# Create a DataFrame from the collected data
df = pd.DataFrame({
    'Product Name': names,
    'Price': prices,
    'Rating': ratings,
    'Ratings Count': ratings_count,
    'Reviews Count': reviews_count,
    'RAM (GB)': ram_list,
    'ROM (GB)': rom_list,
    'Display (inch)': display_list,
    'Battery Capacity (mAh)': battery_list  # Keep battery in mAh
})

# Handle missing data: replace np.nan for missing numerical values
df.fillna({
    'Price': np.nan,
    'Rating': np.nan,
    'Ratings Count': np.nan,
    'Reviews Count': np.nan,
    'RAM (GB)': np.nan,
    'ROM (GB)': np.nan,
    'Display (inch)': np.nan,
    'Battery Capacity (mAh)': np.nan
}, inplace=True)

# Show the DataFrame
print(df)

# Check if DataFrame is empty or not after scraping all pages
if df.empty:
    print("No data was scraped.")
else:
    print(f"Scraped {len(df)} products.")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
                                          Product Name    Price  Rating  \
0             SAMSUNG Galaxy A14 5G (Dark Red, 128 GB)  10999.0     4.2   
1          SAMSUNG Galaxy A14 5G (Light Green, 128 GB)  10999.0     4.2   
2                SAMSUNG Galaxy S23 5G (Cream, 256 GB)  42999.0     4.6   
3            SAMSUNG Galaxy F05 (Twilight Blue, 64 GB)   6499.0     4.2   
4                SAMSUNG Galaxy A14 5G (Black, 128 GB)  10999.0     4.2   
..                                                 ...      ...     ...   
379      SAMSUNG Galaxy M52 5G (Blazing Black, 128 GB)  25999.0     4.2   
380              SAMSUNG Galaxy S23 5G (Green, 128 GB)  37999.0     4

In [26]:
print(df.isnull().sum().sum())

36


In [27]:
print(df.isnull().sum())

Product Name              0
Price                     8
Rating                    4
Ratings Count             4
Reviews Count             4
RAM (GB)                  8
ROM (GB)                  8
Display (inch)            0
Battery Capacity (mAh)    0
dtype: int64


In [31]:
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np

# Initialize empty lists to collect data
device_names = []
colors = []
prices = []
ratings = []
ratings_count = []
reviews_count = []
ram_list = []
rom_list = []
display_list = []
battery_list = []

# Function to extract ratings and reviews numbers
def extract_ratings_and_reviews(feedback):
    pattern = r'(\d{1,3}(?:,\d{3})*)\s*Ratings.*&\s*(\d{1,3}(?:,\d{3})*)\s*Reviews'
    match = re.search(pattern, feedback)
    if match:
        number_of_ratings = int(match.group(1).replace(',', ''))
        number_of_reviews = int(match.group(2).replace(',', ''))
        return number_of_ratings, number_of_reviews
    else:
        return None, None

# Loop through pages from 1 to 17
for page in range(1, 18):
    print(f"Scraping page {page}...")

    # Construct the page URL
    url = ("https://www.flipkart.com/search?q=mobiles&as=on&as-show=on&"
           "otracker=AS_Query_TrendingAutoSuggest_1_0_na_na_na&"
           "otracker1=AS_Query_TrendingAutoSuggest_1_0_na_na_na&"
           "as-pos=1&as-type=TRENDING&suggestionId=mobiles&" 
           "requestId=b87edce2-2302-4ed2-87db-5e11533453f2&p%5B%5D=facets.brand%255B%255D%3DSAMSUNG&" 
           "p%5B%5D=facets.availability%255B%255D%3DExclude%2BOut%2Bof%2BStock&page={}").format(page)

    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36'}
    webpage = requests.get(url, headers=headers)

    # Check if request was successful
    if webpage.status_code != 200:
        print(f"Failed to retrieve page {page}. Status code: {webpage.status_code}")
        continue

    soup = BeautifulSoup(webpage.text, 'html.parser')

    for i in soup.find_all('div', class_="tUxRFH"):
        # Extract product name (handle missing data)
        device_name_tag = i.find('div', class_="KzDlHZ")
        
        if device_name_tag:
            product_list = [device_name_tag.text.strip()]  # Wrap in list for iteration

            for product in product_list:
                # Extract Device Name (everything before the first opening parenthesis)
                name_match = re.match(r'(.*?)(?=\s*\()', product)
                device_name = name_match.group(1).strip() if name_match else product.strip()
                device_names.append(device_name)  # Append directly as it's already a string
                
                # Extract Color (text between the opening and closing parenthesis, excluding GB info)
                color_match = re.search(r'\((.*?),\s*\d+\s*GB\)', product)
                color = color_match.group(1).strip() if color_match else None
                colors.append(color)
        else:
            device_names.append(np.nan)  # Handle missing device name
            colors.append(np.nan)  # Handle missing color

        # Extract product price and convert to integer (handle missing data)
        price_tag = i.find('div', class_="Nx9bqj _4b5DiR")
        prices.append(int(price_tag.text.replace('₹', '').replace(',', '').strip()) if price_tag else np.nan)

        # Extract product rating and convert to float (handle missing data)
        rating_tag = i.find('div', class_="XQDdHH")
        ratings.append(float(rating_tag.text.strip()) if rating_tag else np.nan)

        # Extract ratings_before and ratings_after using the function
        feedback_tag = i.find('span', class_="Wphh3N")
        feedback = feedback_tag.text if feedback_tag else None
        
        if feedback:
            rating_value, review_value = extract_ratings_and_reviews(feedback)
        else:
            rating_value, review_value = None, None
        
        ratings_count.append(rating_value)
        reviews_count.append(review_value)

        # Extract RAM, ROM, Display (inch), and Battery Capacity (mAh)
        text_tag = i.find('ul', class_="G4BRas")
        
        if text_tag:
            text = text_tag.text.strip()

            ram_match = re.search(r'(\d+)\s*GB\s*RAM', text)
            rom_match = re.search(r'(\d+)\s*GB\s*ROM', text)
            display_match = re.search(r'\((\d+(\.\d+)?)\s*inch\)', text)
            battery_match = re.search(r'(\d+)\s*(mAh|MB)', text)

            ram = float(ram_match.group(1)) if ram_match else np.nan
            rom = float(rom_match.group(1)) if rom_match else np.nan
            display = float(display_match.group(1)) if display_match else np.nan
            
            battery = int(battery_match.group(1)) if battery_match else np.nan
        else:
            ram, rom, display, battery = np.nan, np.nan, np.nan, np.nan
        
        ram_list.append(ram)
        rom_list.append(rom)
        display_list.append(display)
        battery_list.append(battery)

# Create a DataFrame from the collected data
df = pd.DataFrame({
    'Product Name': device_names,
    'Colours': colors,
    'Price': prices,
    'Rating': ratings,
    'Ratings Count': ratings_count,
    'Reviews Count': reviews_count,
    'RAM (GB)': ram_list,
    'ROM (GB)': rom_list,
    'Display (inch)': display_list,
    'Battery Capacity (mAh)': battery_list  
})

# Handle missing data: replace np.nan for missing numerical values
df.fillna({
    'Price': np.nan,
    'Rating': np.nan,
    'Ratings Count': np.nan,
    'Reviews Count': np.nan,
    'RAM (GB)': np.nan,
    'ROM (GB)': np.nan,
    'Display (inch)': np.nan,
    'Battery Capacity (mAh)': np.nan
}, inplace=True)

# Show the DataFrame
print(df)

# Check if DataFrame is empty or not after scraping all pages
if df.empty:
    print("No data was scraped.")
else:
    print(f"Scraped {len(df)} products.")


Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraping page 16...
Scraping page 17...
                  Product Name  \
0        SAMSUNG Galaxy A14 5G   
1        SAMSUNG Galaxy A14 5G   
2        SAMSUNG Galaxy S23 5G   
3           SAMSUNG Galaxy F05   
4        SAMSUNG Galaxy A14 5G   
..                         ...   
374  SAMSUNG Galaxy Z Fold4 5G   
375      SAMSUNG Galaxy A14 5G   
376      SAMSUNG Galaxy S23 5G   
377                              
378                              

                                               Colours    Price  Rating  \
0                                             Dark Red  10999.0     4.2   
1                                          Light Green  10999.0     4.2   
2                                  

In [32]:
print(df.isnull().sum().sum())

46


In [33]:
print(df.isnull().sum())

Product Name              0
Colours                   8
Price                     4
Rating                    4
Ratings Count             4
Reviews Count             4
RAM (GB)                  9
ROM (GB)                  8
Display (inch)            5
Battery Capacity (mAh)    0
dtype: int64


In [34]:
df.shape

(379, 10)

In [35]:
print(df.isna().sum())

Product Name              0
Colours                   8
Price                     4
Rating                    4
Ratings Count             4
Reviews Count             4
RAM (GB)                  9
ROM (GB)                  8
Display (inch)            5
Battery Capacity (mAh)    0
dtype: int64


### Downloading DataFrame as an Excel file

In [37]:
!pip install openpyxl

In [38]:
df.to_excel('Flipkart - Samsung Mobiles Data.xlsx',header=True, index=False)